In [1]:
import pandas as pd 
import numpy as np 
import re

In [3]:
file_path=(r"C:\Users\abhis\Downloads\archive (16)\dataset.csv")
df = pd.read_csv(file_path).sample(frac=0.01, random_state=42)

In [4]:
df.shape

(64171, 5)

In [5]:
df.head()

,app_id,app_name,review_text,review_score,review_votes
301327,12210,Grand Theft Auto IV: The Complete Edition,Best bowling simulator 2014 10/10 It has good ...,1,1
1662500,226320,Marvel Heroes Omega,Marvel characters? Check. Tons of loot? Check....,1,0
2061157,236450,PAC-MAN Championship Edition DX+,This game while its not the original is defina...,1,0
1171799,218620,PAYDAY 2,"This game ♥♥♥♥ing awesome ,You can be professi...",1,0
1450080,221640,Super Hexagon,"If you are high, play this game. 420/420 would...",1,0


In [6]:
print(df.iloc[1]["review_text"]) 


Marvel characters? Check. Tons of loot? Check. Tons of characters/classes? Check. Interesting events? Check. LFG? GTFO, You can solo this. Marvel fan? Diablo fan? Good this is for you. Not a marvel fan? Still better than D3 (2013).


In [7]:
df['review_score'].value_counts()

review_score
 1    52649
-1    11522
Name: count, dtype: int64

In [8]:
df.shape

(64171, 5)

In [9]:
df.isnull().sum()

app_id             0
app_name        1874
review_text       73
review_score       0
review_votes       0
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64171 entries, 301327 to 714816
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   app_id        64171 non-null  int64 
 1   app_name      62297 non-null  object
 2   review_text   64098 non-null  object
 3   review_score  64171 non-null  int64 
 4   review_votes  64171 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 2.9+ MB


In [11]:
df["review_text"] = df["review_text"].astype(str)
df["review_votes"] = df["review_votes"].astype(str)

df.review_text = df.review_text.apply(lambda s : s.strip())
df1 =df[df["review_score"].notnull()]
df1["review_score"] = np.where(df1["review_score"] == -1,0, df1["review_score"])
df1.head()

,app_id,app_name,review_text,review_score,review_votes
301327,12210,Grand Theft Auto IV: The Complete Edition,Best bowling simulator 2014 10/10 It has good ...,1,1
1662500,226320,Marvel Heroes Omega,Marvel characters? Check. Tons of loot? Check....,1,0
2061157,236450,PAC-MAN Championship Edition DX+,This game while its not the original is defina...,1,0
1171799,218620,PAYDAY 2,"This game ♥♥♥♥ing awesome ,You can be professi...",1,0
1450080,221640,Super Hexagon,"If you are high, play this game. 420/420 would...",1,0


In [12]:
df1["review_score"].value_counts()

review_score
1    52649
0    11522
Name: count, dtype: int64

In [13]:
df1 = df1[df1.review_text != "Early Access Review"]
df1 = df1[~df1.review_text.isin(['nan'])]

df1.drop_duplicates(['review_text', 'review_score'], inplace= True)


In [14]:
df1.head()

,app_id,app_name,review_text,review_score,review_votes
301327,12210,Grand Theft Auto IV: The Complete Edition,Best bowling simulator 2014 10/10 It has good ...,1,1
1662500,226320,Marvel Heroes Omega,Marvel characters? Check. Tons of loot? Check....,1,0
2061157,236450,PAC-MAN Championship Edition DX+,This game while its not the original is defina...,1,0
1171799,218620,PAYDAY 2,"This game ♥♥♥♥ing awesome ,You can be professi...",1,0
1450080,221640,Super Hexagon,"If you are high, play this game. 420/420 would...",1,0


In [15]:
df1.shape

(52933, 5)

In [16]:
import emoji

def extract_emojis(text):
    return "".join(char for char in str(text) if char in emoji.EMOJI_DATA)

df1['emojis_found'] = df1['review_text'].apply(extract_emojis)

# Show only rows that have emojis
df_reviews_with_emojis = df1[df1['emojis_found'] != '']
print(df_reviews_with_emojis[['review_text', 'emojis_found']])

                                               review_text  \
1171799  This game ♥♥♥♥ing awesome ,You can be professi...   
3403956  I hope the game is good, but I CAN'T EVEN STAR...   
1052101  Once a good game, then turned to ♥♥♥♥ by a 'ba...   
2549863  'God, I'm going fast...God, that was close...G...   
2953816  -Buy game during sale -buy it for your friend ...   
...                                                    ...   
257669   Greatest replay simulater of 2017  666/♥♥♥♥ wo...   
1649929  Hewn from the living rock, this is truely Meta...   
5289330  I see potential for this dev team to make even...   
2211688  Beat this game in 4 hours (including time alt ...   
6136286  Its ♥♥♥♥ing anime trash but overall it is a ho...   

                                         emojis_found  
1171799                                          ♥♥♥♥  
3403956                                          ♥♥♥♥  
1052101                                          ♥♥♥♥  
2549863                        

In [17]:
def replace_hearts_with_PAD(text):
    return re.sub(r"[♥]+", ' **** ' ,text)

In [18]:
def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

In [19]:
def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)

In [20]:
def remove_num(texts):
    output = re.sub(r'\d+', '', texts )
    return output

In [21]:
def unify_whitespaces(text):
    cleaned_string = re.sub(' +', ' ', text )
    return cleaned_string

In [22]:
def remove_punctuation(text):
    result = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"',',') )
    return result

In [23]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def remove_stopwords(text):
    doc = nlp(text)
    text = [token.text.lower() for token in doc if token.text.lower() not in STOP_WORDS]
    return ' '.join(text)

In [30]:
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def lemmatization(text):
    """ Lemmatize using spaCy """
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])


In [31]:
import swifter  

def full_cleaning_pipeline(text):
    text = clean(text)
    text = deEmojify(text)
    text = replace_hearts_with_PAD(text)
    text = remove_num(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = unify_whitespaces(text)
    text = lemmatization(text)
    return text

def cleaning(df, review):
    df_processed = df.copy()
    df_processed[review] = df_processed['review_text']
    
    df_processed[review] = df_processed[review].swifter.apply(full_cleaning_pipeline)
    
    return df_processed

df_processed = cleaning(df1, 'review_text_clean')


Pandas Apply: 100%|██████████| 52933/52933 [31:15<00:00, 28.22it/s]  


In [34]:
df_processed.head()

,app_id,app_name,review_text,review_score,review_votes,emojis_found,review_text_clean
301327,12210,Grand Theft Auto IV: The Complete Edition,Best bowling simulator 2014 10/10 It has good ...,1,1,,good bowl simulator good storyline
1662500,226320,Marvel Heroes Omega,Marvel characters? Check. Tons of loot? Check....,1,0,,marvel character check ton loot check ton char...
2061157,236450,PAC-MAN Championship Edition DX+,This game while its not the original is defina...,1,0,,game original definately good rendition PACMAN...
1171799,218620,PAYDAY 2,"This game ♥♥♥♥ing awesome ,You can be professi...",1,0,♥♥♥♥,game e awesome professional Heister fun play f...
1450080,221640,Super Hexagon,"If you are high, play this game. 420/420 would...",1,0,,high play game dank


In [37]:
df_processed['review_text_clean'].iloc[1]

'marvel character check ton loot check ton charactersclasse check interesting event check LFG GTFO solo Marvel fan Diablo fan Good marvel fan well d'

In [42]:
import emoji

def replace_emoji_with_asterisk(text):
    return ''.join('*' if emoji.is_emoji(char) else char for char in text)

text = "I love programming 😍🎉"
cleaned_text = replace_emoji_with_asterisk(text)
print(cleaned_text)      

I love programming **


In [43]:
reviews = df_processed[df_processed.review_score == 0]
reviews = reviews.sample(n = 2000, random_state = 1234)
rev = reviews.review_text_clean.tolist()

In [46]:
print(reviews.shape)

(2000, 7)


In [47]:
reviews.head()

,app_id,app_name,review_text,review_score,review_votes,emojis_found,review_text_clean
4221162,324260,VoidExpanse,The basics are there but this project is still...,0,0,,basic project alpha stage good pro interesting...
4557387,349040,NARUTO SHIPPUDEN: Ultimate Ninja STORM 4,The game is locked at 30 FPS and sometimes goe...,0,0,,game lock FPS go FPS advertise FPS game handle...
1793278,230410,Warframe,"Boy oh BOY, where do I begin with this one? St...",0,0,,boy oh BOY begin Strap folk get little lengthy...
98130,105600,Terraria,It's not minecraft,0,1,,minecraft
2418335,243470,Watch_Dogs,Pros -It works on windows 8/8.1 Cons -LAGG...,0,0,♥♥♥♥,pro work window Cons laggy


In [ ]:
rev

In [51]:
all_sents = []

for intent in rev:
    doc = nlp(intent)
    for sent in doc.sents:
        if len(sent.text.split()) > 5:
            all_sents.append(sent.text)

print(len(all_sents))
rev = all_sents

1905


In [53]:
from sentence_transformers import SentenceTransformer
import tensorflow as tf 

In [55]:
def embed(model, model_type, sentences ):
    if model_type == 'use':
        embeddings  = model(sentences)
    elif model_type == 'sentence transformer':
        embeddings = model.encode(sentences)
        
    return embeddings
        
model = SentenceTransformer('all-mpnet-base-v2', device='cpu' )

embeddings = embed(model, 'sentence transformer', rev)

In [56]:
embeddings.shape

(1905, 768)

In [58]:
from hyperopt import hp

hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(3,32)),
    "n_components": hp.choice('n_components', range(3,32)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,32)),
    "random_state": 42
}

label_lower = 10
label_upper = 100
max_evals = 25